In [24]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
# Define the columns for the dataframe
columns = ['Core Number', 'Architecture', 'Average Inference Time (ms)', 'Power Consumption (mW)', 'DSP Slice Count', 'RAM Count (Block + Ultra RAM)']

# Create a dataframe and populate it
df = pd.DataFrame(columns=columns, data=[
    ['1 Core', 'B512', 149.24702, 3830, 102, 18],
    ['1 Core', 'B800', 102.31090, 3950, 146, 40],
    ['1 Core', 'B1024', 78.97330, 3990, 166, 26],
    ['1 Core', 'B1152', 73.60489, 4050, 198, 44],
    ['1 Core', 'B1600', 54.64798, 4140, 246, 63.5],
    ['1 Core', 'B2304', 39.55908, 4180, 342, 77.5],
    ['1 Core', 'B3136', 30.18162, 4320, 454, 95.5],
    ['1 Core', 'B4096', 21.25033, 4450, 582, 117.5]
])

In [18]:
df

,Core Number,Architecture,Average Inference Time (ms),Power Consumption (mW),DSP Slice Count,RAM Count (Block + Ultra RAM)
0,1 Core,B512,149.24702,3830,102,18.0
1,1 Core,B800,102.31090,3950,146,40.0
2,1 Core,B1024,78.97330,3990,166,26.0
3,1 Core,B1152,73.60489,4050,198,44.0
4,1 Core,B1600,54.64798,4140,246,63.5
5,1 Core,B2304,39.55908,4180,342,77.5
6,1 Core,B3136,30.18162,4320,454,95.5
7,1 Core,B4096,21.25033,4450,582,117.5


In [21]:
# Function to scale each value to a new range from 1 to 10
def scale_to_range(df, new_min=1, new_max=10):
    return df.apply(lambda x: new_min + ((x - x.min()) * (new_max - new_min) / (x.max() - x.min())))

# Apply the scaling to all numeric columns in the DataFrame
numeric_columns = ['Average Inference Time (ms)', 'Power Consumption (mW)', 'DSP Slice Count', 'RAM Count (Block + Ultra RAM)']
df[numeric_columns] = scale_to_range(df[numeric_columns])

In [22]:
df

,Core Number,Architecture,Average Inference Time (ms),Power Consumption (mW),DSP Slice Count,RAM Count (Block + Ultra RAM)
0,1 Core,B512,10.000000,1.000000,1.000,1.000000
1,1 Core,B800,6.699719,2.741935,1.825,2.989950
2,1 Core,B1024,5.058751,3.322581,2.200,1.723618
3,1 Core,B1152,4.681275,4.193548,2.800,3.351759
4,1 Core,B1600,3.348333,5.500000,3.700,5.115578
5,1 Core,B2304,2.287367,6.080645,5.500,6.381910
6,1 Core,B3136,1.627998,8.112903,7.600,8.010050
7,1 Core,B4096,1.000000,10.000000,10.000,10.000000


In [33]:
# Create subplots with 4 rows and 2 columns
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{'type': 'polar'}]*2]*4,
    subplot_titles=df['Architecture'].tolist()  # Set titles to each subplot
)

# Variables for plotting
theta = ['Average Inference Time (ms)', 'Power Consumption (mW)', 'DSP Slice Count', 'RAM Count (Block + Ultra RAM)']

# Loop through each architecture and create a subplot
for index, row in df.iterrows():
    trace = go.Scatterpolar(
        r=row[['Average Inference Time (ms)', 'Power Consumption (mW)', 'DSP Slice Count', 'RAM Count (Block + Ultra RAM)']].tolist(),
        theta=theta,
        fill='toself',
        name=row['Architecture']
    )
    # Calculate which row and column to place the subplot
    row_index = (index // 2) + 1
    col_index = (index % 2) + 1
    fig.add_trace(trace, row=row_index, col=col_index)

# Apply rotation to all polar subplots
for i in range(1, len(df) + 1):
    fig.update_layout(**{
        f'polar{i}': dict(
            angularaxis=dict(rotation=45, direction='counterclockwise'),
            radialaxis=dict(visible=True)
        )
    })

# Update global layout properties
fig.update_layout(
    height=1600,  # Adjust the height to fit all subplots
    template='plotly_dark',
    showlegend=True
)

fig.show()


In [37]:
# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'polar'}, {'type': 'polar'}],
           [{'type': 'polar'}, {'type': 'polar'}]],
    subplot_titles=[
        'Comparison: 1 Core B512 vs B800',
        'Comparison: 1 Core B3136 vs B4096',
        'Comparison: 1 Core B512 vs B4096',
        'Comparison: 1 Core B800 vs B1600'
    ]
)

# Variables for plotting
theta = ['Average Inference Time (ms)', 'Power Consumption (mW)', 'DSP Slice Count', 'RAM Count (Block + Ultra RAM)']

# List of pairs of indices for architectures to compare
indices_pairs = [(0, 1), (6, 7), (0, 7), (1, 4)]

# Loop through pairs and create each subplot
for i, (idx1, idx2) in enumerate(indices_pairs):
    # First architecture
    fig.add_trace(go.Scatterpolar(
        r=df.loc[idx1, theta].tolist(),
        theta=theta,
        fill='toself',
        name=f'{df.loc[idx1, "Architecture"]}'
    ), row=(i // 2) + 1, col=(i % 2) + 1)
    
    # Second architecture
    fig.add_trace(go.Scatterpolar(
        r=df.loc[idx2, theta].tolist(),
        theta=theta,
        fill='toself',
        name=f'{df.loc[idx2, "Architecture"]}'
    ), row=(i // 2) + 1, col=(i % 2) + 1)

# Update layout for all polar plots
for i in range(1, 5):
    fig.update_layout(**{
        f'polar{i}': dict(
            angularaxis=dict(rotation=45, direction='counterclockwise'),
            radialaxis=dict(visible=True)
        )
    })

# Update global layout properties
fig.update_layout(
    height=1200,  # Adjust height to fit all subplots comfortably
    template='plotly_dark',
    showlegend=True,
    legend=dict(orientation="h", x=0.1, y=1.1)  # Adjust legend positioning
)

fig.show()
